# Lesson 2: Scientific Data Formats and Advanced Plotting

Author: Rebekah Esmaili (rebekah@stcnet.com)
 
---

## Lesson Objectives
* You will learn to:
    * Import relevant packages for scientific programming
    * Read netCDF files
    * Creating plots and maps
   

![](img/flowchart.png)


## Importing NetCDF files

NetCDF and HDF are self-describing formats, which are structured binary data files and useful for storing other big datasets. Computationally, it is faster to read in binary-based datasets than text, which needs to be parsed before being stored in a computer’s memory. Because the files are more compact, they are cheaper to store large, long-term satellite data. Furthermore, information about the data ("metadata") can be stored inside the file themselves.

Many environmental dataset names are quite long. However, the dataset name is encoded to give us information about the contents. For example:

```
JRR-AOD_v2r3_j01_s202304220518119_e202304220519346_c202304220600390.nc
```
You can learn several important features of the dataset without opening it:

* Prefix indicates the mission (JRR, for JPSS Risk Reduction)
* Product (Aerosol Optical Depth, or AOD), algorithm version
* Revision number (v2r3)
* Satellite source (j01 for JPSS-1/NOAA-20)
* Start (s), end (e), and creation (c) time, which are each followed by the year, month, day, hour, minute, and seconds (to one decimal place). 

First, import three commonly used packages in Python:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

To begin, you need to first import [xarray](http://xarray.pydata.org/en/stable/io.html) which is tailored to open netCDF4 files and work with large arrays (like numpy and pandas). The [netCDF4 package](https://unidata.github.io/netcdf4-python/netCDF4/index.html) can also be used to import files. The [h5netcdf](https://github.com/h5netcdf/h5netcdf) is useful because it combines features of both netcdf4 and h5py so you can use one reader for two different file types.

In [ ]:
import xarray as xr
import h5netcdf

Use the open_dataset function to import the above dataset. The engine option is used to read the files. Some possible file readers are "netcdf4", "scipy", "pydap", "h5netcdf", "pynio", "cfgrib", "pseudonetcdf", "zarr" but you also must have the packages installed.  

In [ ]:
fname='data/JRR-AOD_v2r3_j01_s202304220518119_e202304220519346_c202304220600390.nc'
aod_file_id = xr.open_dataset(fname, engine='h5netcdf')

If you print the contents of the file_id variable, you will get a long list of the global attributes, variables, dimensions, and much more.

In [ ]:
aod_file_id

The output above is worth inspecting. Inside Jupyter Notebooks, xarray allows you to inspect the file contents. Clicking on the arrows will show a preview of the metadata. Note that you can also use tools like [Panoply](https://www.giss.nasa.gov/tools/panoply/) to inspect the contents of the netCDF file outside of Python.

* ```Dimensions```: The dimensions are named ```Rows``` and ```Columns```, which are respectively 768 and 3200.

* ```Coordinates```: The coordinates are ```Latitude``` and ```Longitude```. These are both two dimensions.

* ```Variables```: This file has only one variable, which is ```AOD550```. Its dimensions are also ```Rows``` and ```Columns```.

Attributes: netCDF4 [CF-1.5 conventions](https://cfconventions.org/). Some of the information that we saw in the file name is also present: this product is the *JPSS Risk Reduction Unique Aerosol Optical Depth* (title) *Level 2* product (processing_level) and the data was collected from the *NOAA-20* (satellite_name) *VIIRS* instrument (instrument_name). The *start* (time_coverage_start) and *end* times (time_coverage_end) metadata fields are consistent with the filename. I recommend that you read netCDF file header contents, especially the first time you are working with new data. 

AOD is a unitless measure of the extinction of solar radiation by particles suspended in the atmosphere. High values of AOD can indicate the presence of dust, smoke, or another air pollutant while low values indicate a cleaner atmosphere. You can learn more about the AOD algorithm on the [NOAA STAR website](https://www.star.nesdis.noaa.gov/goesr/product_aero_aod.php).

Xarray syntax will resemble both ```Pandas``` and ```NumPy```. Unlike numpy, N-D arrays can be labeled. Instead of having to remember index numbers, we can extract elements using their coordinate or variable names.

Below I'll extract three important variables: AOD550, Latitude, and Longitude:

In [ ]:
AOD_550 = aod_file_id['AOD550']
AOD_lat = aod_file_id['Latitude']
AOD_lon = aod_file_id['Longitude']

Let's print AOD_550 below. This variable contains only a portion of the original data array:

In [ ]:
AOD_550.values

Xarray uses NumPy as a dependency so so we can use numpy functions like ```.mean()```. First, we have to make sure it's in the right format. If you check the type of ```AOD_550```, you can see it's a ```numpy.ndarray```.

In [ ]:
type(AOD_550.values)

Xarray handles missing data automatically, so if we do statistics on the array, it will not include them:

In [ ]:
avgAOD = AOD_550.mean()
print(avgAOD)

For practice, we'll import a Sea Surface Temperature (SST) dataset from GOES-18. You can learn more about the SST dataset on the [NOAA STAR website](https://www.star.nesdis.noaa.gov/goesr/product_sst.php).

---
### Exercise 1 Importing netCDF files
1. Open the file ```data/OR_ABI-L2-SSTF-M6_G18_s20231122000211_e20231122059519_c20231122105091.nc``` using the xarray library
2. Print the variable names
3. What are the dimensions?
---

**Solution:**

## Importing HDF files
HDF files are also common in Earth Sciences. HDF5 is a general-purpose data format, while netCDF4 was designed for storing array-oriented scientific data. Tools like xarray can open HDF5 files if you have helper packages also installed (e.g. ```h5netcdf```).

HDF5 files often are organized into groups, which you can think of as subdirectories to organize the data inside the file. Xarray can open grouped data, but it does not support printing out the various structures in the file. We can use the ```h5py``` package to do so. The syntax is slightly different, but the steps are similar.

In [ ]:
import h5py

filename = 'data/SVM16_j01_d20230422_t0516461_e0518106_b28103_c20230422055438143201_oeac_ops.h5'
f = h5py.File(filename, 'r')

We can use the ```visit``` command to find the contents of the HDF5 file. Visit requires us to pass a function, we will use print.

In [ ]:
f.visit(print)

Another way to do this is combining the ```list``` and ```keys()``` commands:

In [ ]:
dset_all = f['Data_Products/VIIRS-M16-SDR']
list(dset_all.keys())

Once we know the groups, we can pass that into xarray using the group argument (```group='All_Data/VIIRS-M16-SDR_All'```). This dataset has "phony" dimensions, which do not correspond to the physical dimensions of the data. This file does not have coordinates (lat/lon or x/y) values because they are stored in a separate file. We won't use this data any further, but it's here to walk through the steps.

In [ ]:
ds_all = xr.open_dataset(filename, engine='h5netcdf', group='All_Data/VIIRS-M16-SDR_All')
ds_all

## Plotting 3-dimensional Data

So far, we have only made line and scatter plots. Matplotlib also supports plotting spatial datasets. However, we often have to perform several array operations to ensure the x, y, and z coordinates are the same shape. Let's work with a Reflected Shortwave Radiation (RSR) dataset in the next example and make a 3D plot. To learn more, visit the [NOAA STAR](https://www.star.nesdis.noaa.gov/goesr/product_sw.php) page. Below is the file that we will import:

```
data/OR_ABI-L2-RSRF-M6_G16_s20231121800204_e20231121809512_c20231121859124.nc
```

In [ ]:
filename = 'data/OR_ABI-L2-RSRF-M6_G16_s20231121800204_e20231121809512_c20231121859124.nc'
abi_L2_RSR = xr.open_dataset(filename, engine='h5netcdf')

Like before, you can inspect the contents by typing the variable name:

In [ ]:
abi_L2_RSR

From the printed information above, we can see the following:

* __Dimensions__: The dimensions are named ```lat``` and ```lon```, both have the size of 652. There are several additional dimensions (number_of_time_bounds, number_of_image_bounds, number_of_LZA_bounds, number_of_SZA_bounds, number_of_wavelength_bounds) that we won't use in this tutorial.

* __Coordinates__: ```lat``` and ```lon```, which are both one dimensional.

* __Data Variables__: Has 39 variables, we'll focus on ```RSR```.

Below you can import the data values into variables:

In [ ]:
lat_rsr = abi_L2_RSR['lat'].values
lon_rsr = abi_L2_RSR.lon.values
rsr = abi_L2_RSR.RSR.values

Let's inspect the shape and see if the data are already formatted for plotting:

In [ ]:
rsr.shape, lat_rsr.shape, lon_rsr.shape

Contour plots and mesh plots are two useful ways of looking at 3-dimensional data. Both plots require the x, y, and z coordinates to have the same 2D grid. 

You can use ```np.meshgrid()``` to project the 1-dimensional x and y coordinates into two dimensions. The function is a little confusing at first, so I'll show a simple example. Suppose you have two simple arrays:

In [ ]:
tmp_x = [1,2]
tmp_y = [3,4,5]

```tmp_x``` has two elements and ```tmp_y``` has three. If you create a mesh of the two variables, there will be two variables, both with 3 rows and 2 columns: 

In [ ]:
np.meshgrid(tmp_x, tmp_y)

In [ ]:
x2d, y2d = np.meshgrid(tmp_x, tmp_y)

In [ ]:
y2d.shape

Returning to the example, below is the meshgrid of the 1-dimensional latitude and longitude coordinates:

In [ ]:
X_rsr, Y_rsr = np.meshgrid(lon_rsr, lat_rsr)

Before plotting, you need to check if all the dimensions match. However, after comparing the shape of co to X_co, you can see that the dimensions are flipped:

In [ ]:
rsr.shape, X_rsr.shape

We've already learned how to use ```plt.subplot()``` to generate the empty figure (```fig```) and axis (```ax```). 

One line 2, you call ```ax.contourf``` and input the X_co, Y_co, and transposed co variables. co acts as a color value, which becomes the third dimension of the plot. You then store this object into a variable ```rsr_plot``` so that you can pass it into ```ax.colorbar``` to map the colors to numeric values.

In [ ]:
# contourf
fig = plt.figure(figsize=[5,5])
ax1 = plt.subplot(111)
rsr_plot = ax1.contourf(X_rsr, Y_rsr, rsr)
fig.colorbar(rsr_plot, orientation='horizontal', ax=ax1)
plt.show()

Like contour plots, mesh plots are also 2-dimensional plots that display 3-dimensions of information using x and y coordinates, and z for a color scale. However, mesh plots do not perform any smoothing and display data as-is on a regular grid. However, since many satellite datasets are swath-based, irregularly spaced data needs to be re-gridded to display it as a mesh grid. In the code block below, let’s compare how the plot looks using pcolormesh command with the previous example using contour. The code below has no other changes to the plot other than the call to the plot type.

In [ ]:
#pcolormesh
fig = plt.figure(figsize=[5,5])
ax = plt.subplot(111)
sst_plot = ax.pcolormesh(X_rsr, Y_rsr, rsr)
fig.colorbar(sst_plot, orientation='horizontal')
plt.show()

You might notice that there is more structure in the mesh plot than in the filled contour. This is useful if you wish to examine fine structures and patterns.

---
### Exercise 2: Plot 3-dimensional data

Plot ```x```, ```y```, and ```SST``` variables from the ```sst_file_id``` file we opened in Example 1.

1. Extract the variables from the file and assign to three variables 
2. Check the dimensions for all variables using ```.shape```.
3. Do you need to generate a meshgrid with ```np.meshgrid()```?
4. Create a contour plot


---
**Solution:**

## Adding maps to plots

The package [Cartopy](https://scitools.org.uk/cartopy/docs/latest/) add mapping functionality to Matplotlib. Cartopy provides an interface to obtain continent, country, and feature details to overlay onto your plot. Furthermore, Cartopy also enables you to convert your data from one map projection to another, which requires a cartesian coordinate system to the map coordinates. Matplotlib natively supports the six mathematical and map projections (Aitoff, Hammer, Lambert, Mollweide, polar, and rectilinear) and combined with Cartopy, data can be transformed to a total of 33 possible projections.

In [ ]:
from cartopy import crs as ccrs

Just like before, we need to convert the 1D lat and lon coordinates to 2D using meshgrid. We can check the shape to ensure all variables have the same dimensions.

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection=ccrs.PlateCarree())

tmp = ax.contourf(AOD_lon, AOD_lat, AOD_550, levels=np.arange(0, 1.1, 0.1))
fig.colorbar(tmp, orientation='horizontal')
# ax.set_extent([100, 180, -35, 0])

ax.coastlines('50m')
plt.show()

In the next example, you can switch from Plate Carrée to Orthographic. You must define the projection twice, once in the ```projection=``` keyword and again in the ```transform=```. In the ```plt.subplot``` line, you must define the to coordinates (```ccrs.Orthographic```), which is how you want to axes to show the data. In the ```ax.scatter``` line, you use the transform keyword argument in scatter to define the from coordinates (Plate Carrée), which are the coordinates that the data formatted for.

__NOTE__: The next plot may take a few minutes to generate.

In [ ]:
fig = plt.figure(figsize=[10,5])
ax = plt.subplot(projection=ccrs.Orthographic(90, 0))
ax.set_global()

ax.contourf(AOD_lon, AOD_lat, AOD_550, transform=ccrs.PlateCarree())

ax.coastlines()
plt.show()

---
### Exercise 3 Adding maps to plots

Using ```lat_rsr```, ```lon_rsr```, and ```rsr``` (which we imported from the ```OR_ABI-L2-RSRF-M6_G18_...``` netCDF file):

1. Create a ```contourf``` plot (same as Exercise 2)
2. Add the coastlines to Plate Carree plot using ```projection=``` option.

---
**Solution**:

## Summary:

You learned:

* How to import netCDF4, a scientific dataset
* Worked with arrays and lists
* How to create simple maps

Next lesson:
* Obtain datasets from remote sources
* Save data into text and binary files, and plots as images